# Quantile mapping
**Coder:** Nadia Sae-Lim <br>
**Date:** &nbsp; 2024-11-22 <br>
**Goal:** 1) Quantile-map ERA5-Land with the Sibinacocha weather observation for lake modeling study.  <br>

In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF
from math import sqrt

## 1. Import data

In [6]:
# READ REANALYSIS
# ERA5-Land reanalysis
era = pd.read_csv(r'C:\Users\nadia\PhD\lakemodeling\Updated_analysis\ERA5-Land\Hourly\ERA5_Land_raw_1981-2023.csv')  
era['datetime'] = pd.to_datetime(era['datetime'], format='%m/%d/%Y %H:%M')  # convert to datetime format
era.head()

,datetime,Year,Month,Day,Hour,t2m,d2m,ssrd,strd,ssrd_cor,strd_cor,u10,v10,wind,sp
0,1980-12-31 21:00:00,1980,12,31,21,272.521958,272.221006,0.0,9.842987e+05,0.0,273.416309,0.146033,0.010733,0.146427,56302.38640
1,1980-12-31 22:00:00,1980,12,31,22,271.982384,271.671908,0.0,1.970816e+06,0.0,273.416309,-0.406450,0.556874,0.689427,56377.10027
2,1980-12-31 23:00:00,1980,12,31,23,271.661146,271.256852,0.0,2.989069e+06,0.0,274.032511,-0.312276,0.814588,0.872393,56416.08142
3,1981-01-01 00:00:00,1981,1,1,0,271.473812,270.974478,0.0,3.922872e+06,0.0,282.848170,-0.365013,0.315478,0.482453,56378.39964
4,1981-01-01 01:00:00,1981,1,1,1,270.603765,270.240306,0.0,4.817407e+06,0.0,259.389553,-0.430167,0.240611,0.492887,56345.26567


In [7]:
# READ OBSERVATION
# Sibinacocha weather station
obs = pd.read_csv(r"C:\Users\nadia\PhD\lakemodeling\Updated_analysis\Observed-data\obs_Sibi_weather_perutime.csv")   
obs['datetime'] = pd.to_datetime(obs['datetime'], format='%m/%d/%Y %H:%M') 
obs.head()

,datetime,Hour,Temp,Precipitation,wind_obs,RH,Dew,Temp_K
0,2016-03-30 20:00:00,20,3.6,0.0,3.5,78,-0.8,276.75
1,2016-03-30 21:00:00,21,3.4,0.0,1.3,72,-2.2,276.55
2,2016-03-30 22:00:00,22,2.4,0.0,2.0,80,-1.6,275.55
3,2016-03-30 23:00:00,23,1.9,0.0,1.6,82,-1.7,275.05
4,2016-03-31 00:00:00,0,1.9,0.0,0.3,80,-2.1,275.05


## 2. Quantile mapping

In [9]:
# MERGE OBSERVED AND REANALYSIS DATAFRAMES (Overlapped ERA5-Land and Sibinacocha weather station)
merged = pd.merge(era[["datetime","t2m","d2m","ssrd","strd","u10","v10","wind"]],obs,how='outer',left_on='datetime', right_on='datetime')   
merged = merged.dropna(axis=0, how='any')
merged['Month'] = merged['datetime'].dt.month  
merged = merged.reset_index()

In [11]:
# PICK VARIABLE
# 1. Relative humidity
#merged['observed'] = merged['RH'] 
#merged['reanalysis'] = 100 - ((merged['t2m'] - merged["d2m"])*5)
#era['reanalysis'] = 100 - ((era['t2m'] - era["d2m"])*5)

# 2. Temperature
merged['observed'] = merged["Temp_K"]
merged['reanalysis'] = merged["t2m"]
era['reanalysis'] = era["t2m"]

# 3. Wind speed
#merged['observed'] = merged['wind_obs'] 
#merged['reanalysis'] = merged['wind']
#era['reanalysis'] = era['wind']


In [28]:
# CALCULATE AND APPLY QUANTILE MAPPING
mod_ecdf_w = ECDF(merged['reanalysis'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)])
mod_ecdf_s = ECDF(merged['reanalysis'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)])

for j in range(len(era.datetime)):
    if (era.at[j,'Month'] <=3) or (era.at[j,'Month'] >= 10):
        p = mod_ecdf_w(era.at[j,'reanalysis']) * 100   
        corr = np.percentile(merged['observed'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p) - np.percentile(
            merged['reanalysis'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p)      
    else:
        p = mod_ecdf_s(era.at[j,'reanalysis']) * 100   
        corr = np.percentile(merged['observed'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p) - np.percentile(
            merged['reanalysis'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p)
    era.at[j,'t2m_qm'] = era.at[j,'reanalysis'] + corr
    
era.head()

,Unnamed: 0,datetime,Year,Month,Day,Hour,t2m,d2m,ssrd,strd,...,strd_cor,u10,v10,wind,sp,reanalysis,t2m_qm,strd_qm,ssrd_qm,wind_qm
0,0,1980-12-31 21:00:00,1980,12,31,21,272.521958,272.221006,0.0,9.842987e+05,...,273.416309,0.146033,0.010733,0.146427,56302.38640,0.0,274.249656,275.049899,-0.001335,-0.000073
1,1,1980-12-31 22:00:00,1980,12,31,22,271.982384,271.671908,0.0,1.970816e+06,...,273.416309,-0.406450,0.556874,0.689427,56377.10027,0.0,273.749841,275.049899,-0.001335,1.199980
2,2,1980-12-31 23:00:00,1980,12,31,23,271.661146,271.256852,0.0,2.989069e+06,...,274.032511,-0.312276,0.814588,0.872393,56416.08142,0.0,273.450051,275.549932,-0.001335,1.900012
3,3,1981-01-01 00:00:00,1981,1,1,0,271.473812,270.974478,0.0,3.922872e+06,...,282.848170,-0.365013,0.315478,0.482453,56378.39964,0.0,273.349767,286.062952,-0.001335,0.499957
4,4,1981-01-01 01:00:00,1981,1,1,1,270.603765,270.240306,0.0,4.817407e+06,...,259.389553,-0.430167,0.240611,0.492887,56345.26567,0.0,272.849976,263.181199,-0.001335,0.499939


In [ ]:
# Export
era[["wind_qm","t2m_qm","RH_qm"]].to_csv("quantile_mapped_ERA5.csv")